In [83]:
pip install "requests[security]"

Note: you may need to restart the kernel to use updated packages.


In [84]:
import os
import numpy as np
import pandas as pd
from mtcnn.mtcnn import MTCNN
import PIL
from PIL import Image, ImageOps
from numpy import asarray
from os import listdir
from os.path import isfile, join
from scipy.spatial.distance import cosine
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from tqdm.notebook import tqdm
import requests
import cv2

In [85]:
PATH = os.path.abspath(os.getcwd())+'/images_db'
PATH

'/Users/sashakenkre/Documents/umich/courses/fall2021/SI650/Project/images_db'

In [86]:
data = pd.read_csv('mp_db.csv')
data.head()

,Unnamed: 0,DocId,subject,name,date_of_birth,place_of_birth,age,sex,height,race,hair,eyeColor,scars_and_marks,publication,details,description,fieldOffices,url,image,page
0,0,1,Kidnappings and Missing Persons,Katelin Michelle Akens,NaN,NaN,19 years old (at time of disappearance),Female,64.0,White,Blonde,Blue,Akens has a tattoo of five butterflies on her ...,2018-02-26,Katelin Michelle Akens was last seen on Decemb...,"December 5, 2015, Partlow, Virgina",richmond,https://www.fbi.gov/wanted/kidnap/katelin-mich...,https://www.fbi.gov/wanted/kidnap/katelin-mich...,2
1,1,2,ViCAP Missing Persons,Caroline Lee Norris,NaN,NaN,60 years old (at time of disappearance),Female,64.0,Black,Gray,NaN,Norris has a tattoo of a cat paw on her breast...,2020-11-20,Caroline Lee Norris was last seen on September...,"September 11, 2019, Miami, Florida",NaN,https://www.fbi.gov/wanted/vicap/missing-perso...,https://www.fbi.gov/wanted/vicap/missing-perso...,3
2,2,3,ViCAP Missing Persons,Jesus De La Cruz,NaN,NaN,6 years old (at time of disappearance),Male,54.0,White (Hispanic),Brown,Brown,NaN,2021-09-28,Jesus de la Cruz was last seen on September 28...,"September 28, 1996, Lynn, Massachusetts",NaN,https://www.fbi.gov/wanted/vicap/missing-perso...,https://www.fbi.gov/wanted/vicap/missing-perso...,3
3,3,4,Kidnappings and Missing Persons,Aranza Maria Ochoa Lopez,"April 17, 2014","Centralia, Washington",NaN,Female,40.0,White (Hispanic),Blonde/light brown (at time of disappearance),Blue,Aranza has pierced ears.,2019-05-06,The FBI's Seattle Field Office is assisting th...,"October 25, 2018, Vancouver, Washington",seattle,https://www.fbi.gov/wanted/kidnap/aranza-maria...,https://www.fbi.gov/wanted/kidnap/aranza-maria...,3
4,4,5,ViCAP Missing Persons,Mitchell Todd Hein,NaN,NaN,Approximately 22 years old,Male,70.0,White,Brown/auburn and naturally curly,Brown,The victim had a major scar on his knee and a ...,2020-12-10,Jefferson County Sheriff’s Office in Colorado ...,"Indio, California, February 1, 1983",NaN,https://www.fbi.gov/wanted/vicap/missing-perso...,https://www.fbi.gov/wanted/vicap/missing-perso...,3


In [87]:
#Get names for images and save image from url
# for i, url in tqdm(enumerate(data.image)):
#     if 'kidnap/' in url:
#         name = url.replace('/', '-').replace('.','-').split("kidnap-")[1].split('-')[:2]
#         result = ''.join(name)
#     elif 'seeking-info/' in url:
#         name = url.replace('/', '-').replace('.','-').split("info-")[1].split('-')[:2]
#         result = ''.join(name)
#     else:
#         name = url.replace('/', '-').replace('.','-').split("persons-")[1].split('-')[:2]
#         result = ''.join(name)
#     with open('image_db/'+result+'.jpeg', 'wb') as f:
#         f.write(requests.get(url).content)

In [88]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    im = Image.open(filename, mode='r')
    if im.mode != 'RGB':
        pixels = np.array(Image.open(filename, mode='r').convert('RGB'))
        
    elif im.mode == 'RGB':
        pixels = np.array(Image.open(filename, mode='r'))
#     pixels = np.array(PIL.ImageEnhance.Color(img).enhance(0))
        
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
 
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    names = [f for f in filenames]
    # extract faces 
    faces = [extract_face(f) for f in tqdm(filenames)]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    
    d = []
    for name, face in zip(names, yhat):
        d.append([name, face])
    return d
    


In [89]:
# embeddings = get_embeddings(hold)
# print(embeddings)

In [90]:
import pickle

# with open('dataset_faces.dat', 'wb') as f:
#     pickle.dump(embeddings, f)

In [91]:
with open('dataset_faces.dat', 'rb') as f:
    all_face_encodings = pickle.load(f)

In [92]:
def get_single_embedding(filename):
    # extract faces 
    face = extract_face(filename)

    # convert into an array of samples
    sample = asarray(face, 'float32')

    # # prepare the face for the model, e.g. center pixels
    sample = preprocess_input(sample, version=2)
    sample
    # # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    model

    img = cv2.resize(sample,(224,224))     # resize image to match model's expected sizing
    img = img.reshape(1,224,224,3)
    # # perform prediction
    yhat = model.predict(img)
    for x in yhat:
        
        d = [filename, x]
    return d

In [93]:
test = get_single_embedding('test/myra.jpeg')
test

['test/myra.jpeg',
 array([0.       , 4.141911 , 3.5368783, ..., 0.       , 0.9080277,
        2.068684 ], dtype=float32)]

In [94]:
all_face_encodings.insert(0, test)

In [95]:
#use cosine similarity to make matches
def is_match(candidate_embedding, thresh=0.5):
    scores = []
    for i in range(len(all_face_encodings)):
        hold = []
    # calculate distance between embeddings
        score = cosine(all_face_encodings[0][1], all_face_encodings[i][1])
        
        if score <= thresh:
            hold.append([all_face_encodings[i][0], score])
        scores.append(hold)
    return scores

In [96]:
#Get matches
print('Positive Tests')

q = is_match(all_face_encodings)
f = []
for x in q:
    for y in x:
        f.append(y)
f

Positive Tests


[['test/myra.jpeg', 0.0],
 ['images/vanessamorales.jpeg', 0.45487719774246216],
 ['images/diamondyvette.jpeg', 0.3312903046607971],
 ['images/alexiss.jpeg', 0.49289804697036743],
 ['images/aranzamaria.jpeg', 0.4873958230018616],
 ['images/tiondaz.jpeg', 0.4949183464050293],
 ['images/rondreizphillips.jpeg', 0.43927836418151855],
 ['images/myralewis.jpeg', 0.376883864402771]]

In [97]:
#Sort the results and get the top 1
result = sorted(f, key = lambda x: x[1], reverse=False)[:4]
result.pop(0)
result[0]

['images/diamondyvette.jpeg', 0.3312903046607971]